In [ ]:
import requests

url = "http://www.manythings.org/anki/fra-eng.zip"
response = requests.get(url)

if response.status_code == 200:
    with open("fra-eng.zip", "wb") as f:
        f.write(response.content)
    print("Download completed successfully.")
else:
    print("Failed to download the file.")


In [1]:
#importing libraries 
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

In [8]:
batch_s = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dimension = 256  # Latent dimensionality of the encoding space.(Dimentionality of RNN)
number_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = "fra-eng\\fra.txt"

In [9]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(number_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [10]:
print('Number of samples:', len(target_texts))
print('Number of unique input tokens:', num_encoder_tokens)# total number unique characters
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length) # length of longest sentence
print('Max sequence length for outputs:', max_decoder_seq_length) # length of longest sentence

Number of samples: 10000
Number of unique input tokens: 70
Number of unique output tokens: 93
Max sequence length for inputs: 14
Max sequence length for outputs: 59


In [11]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)]) # each character is mapped to a number
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

In [12]:
# creating 3 numpy arrays
#1
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
#2
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
#3
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

shape of array = (len(input_texts) X max_encoder_seq_length X num_encoder_tokens)

In [13]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1. # encoder input data
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1. # encoder padding
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1. # decoder input data
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1. #decoder target data
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1. #decoder input padding
    decoder_target_data[i, t:, target_token_index[' ']] = 1. #decoder target padding

In [14]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dimension, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)  #outputs =(state_h = hidden state , state_c= global content, encoder_output= actual output)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c] # encoder_states(encoder vector)= concat (state_h,state_c)

In [15]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dimension, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [16]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [17]:
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_s,
          epochs=epochs,
          validation_split=0.2)

Epoch 1/100
125/125 [==============================] - 34s 255ms/step - loss: 1.1973 - accuracy: 0.7329 - val_loss: 1.0701 - val_accuracy: 0.7135
Epoch 2/100
125/125 [==============================] - 36s 287ms/step - loss: 0.9273 - accuracy: 0.7497 - val_loss: 0.9516 - val_accuracy: 0.7289
Epoch 3/100
125/125 [==============================] - 35s 281ms/step - loss: 0.8387 - accuracy: 0.7679 - val_loss: 0.8697 - val_accuracy: 0.7518
Epoch 4/100
125/125 [==============================] - 38s 307ms/step - loss: 0.7435 - accuracy: 0.7912 - val_loss: 0.7838 - val_accuracy: 0.7782
Epoch 5/100
125/125 [==============================] - 40s 317ms/step - loss: 0.6738 - accuracy: 0.8070 - val_loss: 0.7172 - val_accuracy: 0.7886
Epoch 6/100
125/125 [==============================] - 37s 300ms/step - loss: 0.6326 - accuracy: 0.8166 - val_loss: 0.6822 - val_accuracy: 0.8007
Epoch 7/100
125/125 [==============================] - 37s 298ms/step - loss: 0.6041 - accuracy: 0.8233 - val_loss: 0.6715 -

Epoch 57/100
125/125 [==============================] - 42s 335ms/step - loss: 0.2445 - accuracy: 0.9266 - val_loss: 0.4384 - val_accuracy: 0.8779
Epoch 58/100
125/125 [==============================] - 45s 363ms/step - loss: 0.2414 - accuracy: 0.9277 - val_loss: 0.4469 - val_accuracy: 0.8766
Epoch 59/100
125/125 [==============================] - 42s 337ms/step - loss: 0.2382 - accuracy: 0.9283 - val_loss: 0.4358 - val_accuracy: 0.8784
Epoch 60/100
125/125 [==============================] - 42s 336ms/step - loss: 0.2346 - accuracy: 0.9296 - val_loss: 0.4380 - val_accuracy: 0.8788
Epoch 61/100
125/125 [==============================] - 42s 338ms/step - loss: 0.2316 - accuracy: 0.9303 - val_loss: 0.4446 - val_accuracy: 0.8770
Epoch 62/100
125/125 [==============================] - 40s 324ms/step - loss: 0.2280 - accuracy: 0.9315 - val_loss: 0.4430 - val_accuracy: 0.8779
Epoch 63/100
125/125 [==============================] - 42s 336ms/step - loss: 0.2251 - accuracy: 0.9321 - val_loss: 0

In [18]:
# Save model
model.save('eng_fra.keras')

In [19]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dimension,))
decoder_state_input_c = Input(shape=(latent_dimension,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [20]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [21]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [22]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    
    print('-')
    print('Input sentence:', input_texts[seq_index] )
    print('Decoded sentence:', decoded_sentence)


1/1 [==============================] - 0s 20ms/step
-
Input sentence: Go.
Decoded sentence: Achète aus.

1/1 [==============================] - 0s 15ms/step
-
Input sentence: Go.
Decoded sentence: Achète aus.

1/1 [==============================] - 0s 17ms/step
-
Input sentence: Go.
Decoded sentence: Achète aus.

1/1 [==============================] - 0s 15ms/step
-
Input sentence: Go.
Decoded sentence: Achète aus.

1/1 [==============================] - 0s 16ms/step
-
Input sentence: Hi.
Decoded sentence: Salut !

1/1 [==============================] - 0s 16ms/step
-
Input sentence: Hi.
Decoded sentence: Salut !

1/1 [==============================] - 0s 24ms/step
-
Input sentence: Run!
Decoded sentence: Fuyons !

1/1 [==============================] - 0s 27ms/step
-
Input sentence: Run!
Decoded sentence: Fuyons !

1/1 [==============================] - 0s 20ms/step
-
Input sentence: Run!
Decoded sentence: Fuyons !

1/1 [==============================] - 0s 26ms/step
-
Input sentence:

1/1 [==============================] - 0s 20ms/step
-
Input sentence: Run!
Decoded sentence: Fuyons !

1/1 [==============================] - 0s 23ms/step
-
Input sentence: Run.
Decoded sentence: Fuyons !

1/1 [==============================] - 0s 18ms/step
-
Input sentence: Run.
Decoded sentence: Fuyons !

1/1 [==============================] - 0s 26ms/step
-
Input sentence: Run.
Decoded sentence: Fuyons !

1/1 [==============================] - 0s 25ms/step
-
Input sentence: Run.
Decoded sentence: Fuyons !

1/1 [==============================] - 0s 16ms/step
-
Input sentence: Run.
Decoded sentence: Fuyons !

1/1 [==============================] - 0s 14ms/step
-
Input sentence: Run.
Decoded sentence: Fuyons !

1/1 [==============================] - 0s 18ms/step
-
Input sentence: Run.
Decoded sentence: Fuyons !

1/1 [==============================] - 0s 28ms/step
-
Input sentence: Run.
Decoded sentence: Fuyons !

1/1 [==============================] - 0s 17ms/step
-
Input sentence: Who

1/1 [==============================] - 0s 29ms/step
-
Input sentence: Wow!
Decoded sentence: Cas-cu santimon !

1/1 [==============================] - 0s 9ms/step
-
Input sentence: Duck!
Decoded sentence: Baisse-toi.

1/1 [==============================] - 0s 15ms/step
-
Input sentence: Duck!
Decoded sentence: Baisse-toi.

1/1 [==============================] - 0s 19ms/step
-
Input sentence: Duck!
Decoded sentence: Baisse-toi.

1/1 [==============================] - 0s 28ms/step
-
Input sentence: Fire!
Decoded sentence: Au feu !

1/1 [==============================] - 0s 17ms/step
-
Input sentence: Help!
Decoded sentence: Accondenan.

1/1 [==============================] - 0s 17ms/step
-
Input sentence: Hide.
Decoded sentence: Attendiz.

1/1 [==============================] - 0s 17ms/step
-
Input sentence: Hide.
Decoded sentence: Attendiz.

1/1 [==============================] - 0s 15ms/step
-
Input sentence: Jump!
Decoded sentence: Saute.

1/1 [==============================] - 0s 23m

1/1 [==============================] - 0s 20ms/step
-
Input sentence: Stop!
Decoded sentence: Arrête-toi !

1/1 [==============================] - 0s 18ms/step
-
Input sentence: Wait!
Decoded sentence: Attendez.

1/1 [==============================] - 0s 16ms/step
-
Input sentence: Wait!
Decoded sentence: Attendez.

1/1 [==============================] - 0s 21ms/step
-
Input sentence: Wait!
Decoded sentence: Attendez.

1/1 [==============================] - 0s 20ms/step
-
Input sentence: Wait.
Decoded sentence: Attendez.

1/1 [==============================] - 0s 16ms/step
-
Input sentence: Wait.
Decoded sentence: Attendez.

1/1 [==============================] - 0s 19ms/step
-
Input sentence: Wait.
Decoded sentence: Attendez.

1/1 [==============================] - 0s 22ms/step
-
Input sentence: Wait.
Decoded sentence: Attendez.

1/1 [==============================] - 0s 18ms/step
-
Input sentence: Begin.
Decoded sentence: Ouvour !

1/1 [==============================] - 0s 26ms/step


1/1 [==============================] - 0s 15ms/step
-
Input sentence: Hello!
Decoded sentence: Salut.

1/1 [==============================] - 0s 20ms/step
-
Input sentence: Hello!
Decoded sentence: Salut.

1/1 [==============================] - 0s 15ms/step
-
Input sentence: Hello.
Decoded sentence: Salut !

1/1 [==============================] - 0s 19ms/step
-
Input sentence: Hello.
Decoded sentence: Salut !

1/1 [==============================] - 0s 16ms/step
-
Input sentence: Hello.
Decoded sentence: Salut !

1/1 [==============================] - 0s 26ms/step
-
Input sentence: Hello.
Decoded sentence: Salut !

1/1 [==============================] - 0s 27ms/step
-
Input sentence: I see.
Decoded sentence: Je comprends.

1/1 [==============================] - 0s 18ms/step
-
Input sentence: I see.
Decoded sentence: Je comprends.

1/1 [==============================] - 0s 24ms/step
-
Input sentence: I try.
Decoded sentence: Je rois.

1/1 [==============================] - 0s 20ms/step
-

1/1 [==============================] - 0s 20ms/step
-
Input sentence: I won.
Decoded sentence: Je l'ai emporté.

1/1 [==============================] - 0s 16ms/step
-
Input sentence: Oh no!
Decoded sentence: À qui ?

1/1 [==============================] - 0s 29ms/step
-
Input sentence: Relax.
Decoded sentence: Détends-toi.

1/1 [==============================] - 0s 20ms/step
-
Input sentence: Relax.
Decoded sentence: Détends-toi.

1/1 [==============================] - 0s 22ms/step
-
Input sentence: Relax.
Decoded sentence: Détends-toi.

1/1 [==============================] - 0s 15ms/step
-
Input sentence: Relax.
Decoded sentence: Détends-toi.

1/1 [==============================] - 0s 19ms/step
-
Input sentence: Relax.
Decoded sentence: Détends-toi.

1/1 [==============================] - 0s 14ms/step
-
Input sentence: Relax.
Decoded sentence: Détends-toi.

1/1 [==============================] - 0s 22ms/step
-
Input sentence: Relax.
Decoded sentence: Détends-toi.

1/1 [===============

1/1 [==============================] - 0s 21ms/step
-
Input sentence: Relax.
Decoded sentence: Détends-toi.

1/1 [==============================] - 0s 20ms/step
-
Input sentence: Relax.
Decoded sentence: Détends-toi.

1/1 [==============================] - 0s 23ms/step
-
Input sentence: Relax.
Decoded sentence: Détends-toi.

1/1 [==============================] - 0s 23ms/step
-
Input sentence: Smile.
Decoded sentence: Souriez !

1/1 [==============================] - 0s 15ms/step
-
Input sentence: Smile.
Decoded sentence: Souriez !

1/1 [==============================] - 0s 18ms/step
-
Input sentence: Smile.
Decoded sentence: Souriez !

1/1 [==============================] - 0s 13ms/step
-
Input sentence: Sorry?
Decoded sentence: Intaie-toi.

1/1 [==============================] - 0s 21ms/step
-
Input sentence: Attack!
Decoded sentence: Attentez-le.

1/1 [==============================] - 0s 14ms/step
-
Input sentence: Attack!
Decoded sentence: Attentez-le.

1/1 [======================

1/1 [==============================] - 0s 28ms/step
-
Input sentence: Buy it.
Decoded sentence: Achetez-le !

1/1 [==============================] - 0s 21ms/step
-
Input sentence: Buy it.
Decoded sentence: Achetez-le !

1/1 [==============================] - 0s 25ms/step
-
Input sentence: Buy it.
Decoded sentence: Achetez-le !

1/1 [==============================] - 0s 20ms/step
-
Input sentence: Buy it.
Decoded sentence: Achetez-le !

1/1 [==============================] - 0s 26ms/step
-
Input sentence: Cheers!
Decoded sentence: À viaine.

1/1 [==============================] - 0s 15ms/step
-
Input sentence: Cheers!
Decoded sentence: À viaine.

1/1 [==============================] - 0s 18ms/step
-
Input sentence: Cheers!
Decoded sentence: À viaine.

1/1 [==============================] - 0s 22ms/step
-
Input sentence: Cheers!
Decoded sentence: À viaine.

1/1 [==============================] - 0s 28ms/step
-
Input sentence: Eat it.
Decoded sentence: Attendez.

1/1 [====================

1/1 [==============================] - 0s 14ms/step
-
Input sentence: Get up.
Decoded sentence: Décuerpissez.

1/1 [==============================] - 0s 15ms/step
-
Input sentence: Go now.
Decoded sentence: Va tor bien !

1/1 [==============================] - 0s 26ms/step
-
Input sentence: Go now.
Decoded sentence: Va tor bien !

1/1 [==============================] - 0s 25ms/step
-
Input sentence: Go now.
Decoded sentence: Va tor bien !

1/1 [==============================] - 0s 29ms/step
-
Input sentence: Got it!
Decoded sentence: Va te faire foutre !

